In [2]:
library(Seurat)
library(Signac)
library(SeuratWrappers)
library(RENIN)
library(tidyverse)
library(chromVARmotifs)
library(BSgenome.Hsapiens.UCSC.hg19)

Attaching SeuratObject

Attaching sp


Attaching package: ‘Signac’


The following object is masked from ‘package:Seurat’:

    FoldChange


── Attaching packages ─────────────────────────────────────────────────────────────────────────────── tidyverse 1.3.2 ──
✔ ggplot2 3.4.1     ✔ purrr   0.3.4
✔ tibble  3.1.8     ✔ dplyr   1.0.9
✔ tidyr   1.2.0     ✔ stringr 1.4.0
✔ readr   2.1.2     ✔ forcats 0.5.1
── Conflicts ────────────────────────────────────────────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
✖ purrr::reduce() masks Signac::reduce()


Warning message:
“no function found corresponding to methods exports from ‘BSgenome’ for: ‘releaseName’”
Loading required package: BSgenome

Loading required package: BiocGenerics

Loading required package: parallel


Attaching package: ‘BiocGenerics’


The following objects are masked from ‘package:parallel’:

    clusterApply, clusterApplyLB, cluste

## define the two ident to compare in FindMarkers function

In [3]:
FindMarkers_ident_1 = "PT"
FindMarkers_ident_2 = "PT_VCAM1"

In [4]:
de_gene_file = "../../processed_data/RENIN/de_genes/446267_cells_RNA/PT_rank_genes_groups_PT_vs_PTVCAM1_20230425.RData"

In [5]:
processed_dir = file.path("..", "..", "processed_data", "RENIN", "by_celltypes", paste0(gsub(pattern = "\\/", "_", paste(c(FindMarkers_ident_1,FindMarkers_ident_2), collapse = "_vs_"))))
processed_dir
dir.create(processed_dir, recursive = T, showWarnings = F)

[1] "../../processed_data/RENIN/by_celltypes/PT_vs_PT_VCAM1"

### load previously calculated image which includes expr_mat, peak_mat, novaseq.sub. This could save time for dealing with new de_gene for different clusters while sharing the same novaseq.sub object

In [6]:
Sys.time()
load("../../processed_data/RENIN/RENIN_324701_cells_preprocess.RData")
Sys.time()

[1] "2023-06-30 23:04:20 CDT"

[1] "2023-06-30 23:09:11 CDT"

In [7]:
novaseq.sub
table(novaseq.sub$celltype5_rna)

An object of class Seurat 
237522 features across 324701 samples within 2 assays 
Active assay: peaks (189184 features, 189184 variable features)
 1 other assay present: SCT
 6 dimensional reductions calculated: pca, harmony_RNA, lsi, harmony_peaks, umap.peaks, WNN.UMAP


      JGA       POD       PEC        PT PT_dediff  PT_VCAM1       tL1       tL2 
     2465      3179      1239     64880     29494      8445     25438      9029 
   tL-TAL      TAL1      TAL2      TAL3       DCT       CNT       PC1       PC2 
    15485     29910     19724      8900     14150     13197     16140      4049 
      ICA       ICB      Uro1      Uro2      ENDO      SMC1      SMC2      SMC3 
    10579      4120      3210       682     12847      4272       156        86 
     Fib1      Fib2      Fib3        Ma        BT 
     3786      7191      3095      3673      5280 

In [8]:
ls()

[1] "de_gene_file"                     "expr_mat"                        
 [3] "FindMarkers_ident_1"              "FindMarkers_ident_2"             
 [5] "level.novaseq"                    "level.novaseq.renal_region_new"  
 [7] "level.novaseq.rna"                "novaseq.sub"                     
 [9] "palette.novaseq"                  "palette.novaseq.renal_region_new"
[11] "palette.novaseq.rna"              "peak_mat"                        
[13] "processed_dir"

### remove the following variables because they will be regenerated for new comparison

In [9]:
rm(list = c("peak_results", 'aen_lists', 'de_genes_BK', 'gene_list'))

Warning message in rm(list = c("peak_results", "aen_lists", "de_genes_BK", "gene_list")):
“object 'peak_results' not found”
Warning message in rm(list = c("peak_results", "aen_lists", "de_genes_BK", "gene_list")):
“object 'aen_lists' not found”
Warning message in rm(list = c("peak_results", "aen_lists", "de_genes_BK", "gene_list")):
“object 'de_genes_BK' not found”
Warning message in rm(list = c("peak_results", "aen_lists", "de_genes_BK", "gene_list")):
“object 'gene_list' not found”


In [10]:
ls()

[1] "de_gene_file"                     "expr_mat"                        
 [3] "FindMarkers_ident_1"              "FindMarkers_ident_2"             
 [5] "level.novaseq"                    "level.novaseq.renal_region_new"  
 [7] "level.novaseq.rna"                "novaseq.sub"                     
 [9] "palette.novaseq"                  "palette.novaseq.renal_region_new"
[11] "palette.novaseq.rna"              "peak_mat"                        
[13] "processed_dir"

In [11]:
Idents(novaseq.sub) = novaseq.sub$celltype5_rna
head(Idents(novaseq.sub))

R1.034,R2.082,R3.082,P1.76,B3 R1.047,R2.073,R3.003,P1.65,B2 
                          tL1                           PEC 
R1.025,R2.080,R3.017,P1.66,B2 R1.056,R2.038,R3.008,P1.48,B2 
                    PT_dediff                          TAL1 
R1.089,R2.082,R3.024,P1.14,B1 R1.031,R2.072,R3.075,P1.78,B3 
                          CNT                          TAL2 
29 Levels: JGA POD PEC PT PT_dediff PT_VCAM1 tL1 tL2 tL-TAL TAL1 TAL2 ... BT

In [12]:
mpt <- subset(novaseq.sub, celltype5_rna %in% c(FindMarkers_ident_1, FindMarkers_ident_2))
mpt
table(mpt$celltype5_rna)

An object of class Seurat 
237522 features across 73325 samples within 2 assays 
Active assay: peaks (189184 features, 189184 variable features)
 1 other assay present: SCT
 6 dimensional reductions calculated: pca, harmony_RNA, lsi, harmony_peaks, umap.peaks, WNN.UMAP


      JGA       POD       PEC        PT PT_dediff  PT_VCAM1       tL1       tL2 
        0         0         0     64880         0      8445         0         0 
   tL-TAL      TAL1      TAL2      TAL3       DCT       CNT       PC1       PC2 
        0         0         0         0         0         0         0         0 
      ICA       ICB      Uro1      Uro2      ENDO      SMC1      SMC2      SMC3 
        0         0         0         0         0         0         0         0 
     Fib1      Fib2      Fib3        Ma        BT 
        0         0         0         0         0 

## load 446267_cells_RNA de_gene data frame

In [13]:
load(de_gene_file)

In [14]:
ls()

[1] "de_gene_file"                     "de_genes"                        
 [3] "expr_mat"                         "FindMarkers_ident_1"             
 [5] "FindMarkers_ident_2"              "level.novaseq"                   
 [7] "level.novaseq.renal_region_new"   "level.novaseq.rna"               
 [9] "mpt"                              "novaseq.sub"                     
[11] "palette.novaseq"                  "palette.novaseq.renal_region_new"
[13] "palette.novaseq.rna"              "peak_mat"                        
[15] "processed_dir"

In [15]:
rownames(de_genes) = de_genes$names
de_genes_BK = de_genes
dim(de_genes)

[1] 48555     5

In [16]:
### keep DEGs with pvals_adj<0.05
p_val_cutoff = 0.05
peak_assay = "peaks"
de_genes <- de_genes[which(de_genes$pvals_adj < p_val_cutoff), ]
DefaultAssay(novaseq.sub) <- peak_assay
DefaultAssay(mpt) <- peak_assay

dim(de_genes)

[1] 1765    5

In [17]:
Sys.time()
# gene_coords <- Signac:::CollapseToLongestTranscript(Annotation(novaseq.sub))
gene_coords <- Signac:::CollapseToLongestTranscript(Annotation(mpt))
Sys.time()

[1] "2023-06-30 23:10:02 CDT"

[1] "2023-06-30 23:10:06 CDT"

In [18]:
de_genes <- de_genes[which(rownames(de_genes) %in% gene_coords$gene_name), ]
dim(de_genes)
gene_list = rownames(de_genes)

[1] 1471    5

## Next, make two de_gene sublists: gene_list_neg, gene_list_pos

In [19]:
gene_list_neg = rownames(de_genes)[de_genes$logfoldchanges < 0]
print(length(gene_list_neg))
gene_list_pos = rownames(de_genes)[de_genes$logfoldchanges > 0]
print(length(gene_list_pos))

[1] 1274
[1] 197


In [20]:
# de_genes[order(de_genes$logfoldchanges, decreasing = T),]

## run_peak_aen

In [21]:
Sys.time()

options(future.globals.maxSize = 8000 * 1024^2)

peak_results <- run_peak_aen(novaseq.sub, expr_mat, peak_mat, gene_list, lambda2 = 0.5, max_distance = 5e+05, num_bootstraps = 100)

aen_lists <- make_aen_lists(peak_results)

Sys.time()

[1] "2023-06-30 23:10:06 CDT"

Loading required package: gcdnet


Attaching package: ‘gcdnet’


The following object is masked from ‘package:stats4’:

    coef


The following objects are masked from ‘package:stats’:

    coef, predict


Loading required package: future


Attaching package: ‘future’


The following object is masked from ‘package:rtracklayer’:

    values


The following object is masked from ‘package:GenomicRanges’:

    values


The following object is masked from ‘package:IRanges’:

    values


The following object is masked from ‘package:S4Vectors’:

    values


Loading required package: future.apply



[1] "Omitted (mitochondrial) genes with no genomic peaks: PCDH11Y"  
[2] "Omitted (mitochondrial) genes with no genomic peaks: SMN1"     
[3] "Omitted (mitochondrial) genes with no genomic peaks: PCDH11X"  
[4] "Omitted (mitochondrial) genes with no genomic peaks: LINC00278"
[1] "AEN completed in 11.4115601181984"


[1] "2023-06-30 23:22:01 CDT"

### prepare meta cells for mpt object

In [22]:
expr_mat_2 <- prepare_pseudocell_matrix(mpt, 
                                      assay = "SCT", 
                                      slot = "data", 
                                      cells_per_partition = 10, 
                                      find_neighbors = FALSE,
                                      reduction1 = "harmony_peaks",
                                      reduction2 = "harmony_RNA",
                                      dim_list = list(1:50, 1:50),
                                      k.nn = 5,
                                      seed = 489284)

Loading required package: Matrix


Attaching package: ‘Matrix’


The following object is masked from ‘package:S4Vectors’:

    expand


The following objects are masked from ‘package:tidyr’:

    expand, pack, unpack


Loading required package: VISION

Loading required package: SingleCellExperiment

Loading required package: SummarizedExperiment

Loading required package: MatrixGenerics

Loading required package: matrixStats


Attaching package: ‘matrixStats’


The following object is masked from ‘package:dplyr’:

    count



Attaching package: ‘MatrixGenerics’


The following objects are masked from ‘package:matrixStats’:

    colAlls, colAnyNAs, colAnys, colAvgsPerRowSet, colCollapse,
    colCounts, colCummaxs, colCummins, colCumprods, colCumsums,
    colDiffs, colIQRDiffs, colIQRs, colLogSumExps, colMadDiffs,
    colMads, colMaxs, colMeans2, colMedians, colMins, colOrderStats,
    colProds, colQuantiles, colRanges, colRanks, colSdDiffs, colSds,
    colSums2, colTabulates, colVarDif

In [23]:
length(gene_list)
sum(gene_list %in% colnames(expr_mat_2))

[1] 1471

[1] 1471

### run_tf_aen

In [24]:
Sys.time()

tf_results <- run_tf_aen(mpt, expr_mat_2, peak_results, gene_list, lambda2 = 0.5)

regulator_tf_names <- unlist(novaseq.sub@assays$peaks@motifs@motif.names)

regulator_tf_names <- regulator_tf_names[which(regulator_tf_names %in% rownames(GetAssayData(novaseq.sub, assay = "SCT")))]

centrality_rankings <- rank_tfs_by_centrality(tf_results, novaseq.sub)

Sys.time()

[1] "2023-06-30 23:41:54 CDT"

[1] "Genes without any candidate TFs--most likely no linked peaks or promoter region peaks: BTBD2" 
[2] "Genes without any candidate TFs--most likely no linked peaks or promoter region peaks: NBPF20"
[3] "Genes without any candidate TFs--most likely no linked peaks or promoter region peaks: AGAP6" 
[4] "Genes without any candidate TFs--most likely no linked peaks or promoter region peaks: SMN2"  
[5] "Genes without any candidate TFs--most likely no linked peaks or promoter region peaks: DTX2"  
[6] "Genes without any candidate TFs--most likely no linked peaks or promoter region peaks: GIGYF1"
[7] "Genes without any candidate TFs--most likely no linked peaks or promoter region peaks: FAM21B"
[8] "Genes without any candidate TFs--most likely no linked peaks or promoter region peaks: CBWD7" 
[9] "Genes without any candidate TFs--most likely no linked peaks or promoter region peaks: CDH18" 
[1] "AEN completed in 43.0119196335475"


Loading required package: igraph


Attaching package: ‘igraph’


The following objects are masked from ‘package:future’:

    %->%, %<-%


The following objects are masked from ‘package:rtracklayer’:

    blocks, path


The following object is masked from ‘package:Biostrings’:

    union


The following object is masked from ‘package:XVector’:

    path


The following object is masked from ‘package:GenomicRanges’:

    union


The following object is masked from ‘package:IRanges’:

    union


The following object is masked from ‘package:S4Vectors’:

    union


The following objects are masked from ‘package:BiocGenerics’:

    normalize, path, union


The following objects are masked from ‘package:dplyr’:

    as_data_frame, groups, union


The following objects are masked from ‘package:purrr’:

    compose, simplify


The following object is masked from ‘package:tidyr’:

    crossing


The following object is masked from ‘package:tibble’:

    as_data_frame


The following objects a

[1] "2023-07-01 00:26:17 CDT"

In [25]:
print(Sys.time())
    
print("centrality_rankings - Betweenness:")
print(head(centrality_rankings$Betweenness,20))
print("centrality_rankings - PageRank:")
print(head(centrality_rankings$PageRank,20))

print(Sys.time())
print("calculating rank_tfs...")
tf_ranks = rank_tfs(tf_results, 
                    negative_genes = rownames(de_genes_BK)[which(de_genes_BK$logfoldchanges < 0)], 
                    seurat = mpt, 
                    num_cores =  1
    )
print(Sys.time())

[1] "2023-07-01 00:26:17 CDT"
[1] "centrality_rankings - Betweenness:"
     GLIS3      DNMT1      ESRRG      ZFHX3      NR1H4     BCL11A      RUNX1 
36029.0500 31428.9310 23401.7214 21416.0071 12745.1262  8190.6881  8019.9429 
       MAF       RARB       HBP1     ZNF75D        JUN       TFEC       NFYC 
 3937.5095  3859.3167  2534.2667  2159.7595  1758.4500  1402.6333  1146.1833 
    POU6F2      FOXP1       ELF3      FOXN3     ZBTB7C       ATF1 
 1104.5024   927.0000   660.4929   659.2833   483.0000   436.8667 
[1] "centrality_rankings - PageRank:"
      TCF12        NFIB        PAX8       GLIS3       ESRRG        SOX6 
0.019624636 0.018442626 0.017476904 0.014657995 0.010693771 0.010601180 
      NR1H4        RORA        ETV6       HNF4A       STAT1        KLF6 
0.010144044 0.009840880 0.009688696 0.009456987 0.009216004 0.008851263 
      PPARA       NFAT5       MEF2A       STAT3       TEAD1        NFIA 
0.008255799 0.007980012 0.007341864 0.007114467 0.006922345 0.006670303 
     AR

In [26]:
head(tf_ranks,20)
tail(tf_ranks,20)

,TF_name,Score,SE
,<chr>,<dbl>,<dbl>
PPARA,PPARA,1.8817809,0.04537664
HNF4A,HNF4A,1.7123379,0.03380345
ARNTL2,ARNTL2,1.5350156,0.07229693
DNMT1,DNMT1,1.4133519,0.03961257
ESRRG,ESRRG,0.6965156,0.08920942
SOX5,SOX5,0.6240492,0.03170411
BCL11A,BCL11A,0.5758600,0.03024291
ZFHX3,ZFHX3,0.5690468,0.02959775
RUNX1,RUNX1,0.5371439,0.03042104


,TF_name,Score,SE
,<chr>,<dbl>,<dbl>
HIVEP2,HIVEP2,-0.3710029,0.01903864
FOXP2,FOXP2,-0.4307060,0.02562943
TEAD1,TEAD1,-0.4641859,0.04508974
CREB5,CREB5,-0.4690078,0.01603769
HIF1A,HIF1A,-0.4900706,0.02270890
ZNF148,ZNF148,-0.4958536,0.02928001
MITF,MITF,-0.5163732,0.02284001
KLF6,KLF6,-0.5183941,0.01630609
STAT3,STAT3,-0.5348554,0.02898480


In [27]:
save(list=c("gene_list", "aen_lists", "de_genes_BK", "tf_results", "tf_ranks", "regulator_tf_names", "centrality_rankings", "peak_results"), 
     file = file.path(processed_dir, paste0("RENIN_324701_cells_", FindMarkers_ident_1, "_vs_", FindMarkers_ident_2, ".RData")), compress = T)

In [28]:
Sys.time()

[1] "2023-07-01 00:29:47 CDT"